<a href="https://colab.research.google.com/github/dbtronics/COMP551---Miniproject-3-Classification-of-Image-Data-/blob/main/Group_18_Miniproject_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

import pdb

# MLP
Multilayer Perceptron Implementation

In [10]:
# logistic = lambda z: 1./ (1 + np.exp(-z))
# softmax = lambda z: np.exp(-z) / np.sum(np.exp(-z), axis = -1)[:, None]

# class MLP:

    
#   def __init__(self, M = 64):
#     self.M = M
            
#   def fit(self, x, y, optimizer, hidden_layer=1):
#     N,D = x.shape
#     C = np.max(y)+1
#     self.layers = hidden_layer
#     y_realprob = np.zeros((N, C))
#     for i in range(N): y_realprob[i, y[i]] = 1
#     # START: Condition to have 0, 1, 2 layers
#     # gradient(x,y,params, hidden_layers)
#     def gradient_0layer(x, y, params):
#       w = params
#       yh = softmax(np.dot(x, w))
#       dy = yh - y_realprob # Squared residual loss derivative
#       # dw = np.dot(x.T, dy)
#       dw = (x.T @ dy)/N
#       dparams = [dw]


#     def gradient_1layer(x, y, params):
#       v, w = params
#       z = logistic(np.dot(x, v)) #N x M
#       yh = softmax(np.dot(z, w))#N x C
#       # print(yh[i, np.argmax(yh[i,:])] for i in range(10))
#       # print(y[:10])
#       dy = yh - y_realprob #N x C
#       dw = np.dot(z.T, dy)/N #M x C
#       dz = np.dot(dy, w.T) #N x M
#       # print(dz.shape)
#       dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
#       dparams = [dv, dw]
#       return dparams
    
#     def gradient_2layer(x, y, params):
#       return None
#     gradient = [gradient_0layer, gradient_1layer, gradient_2layer]
        
#     # params0 = [np.random.randn(self.M) for i in range(hidden_layer+1)]
#     if(hidden_layer==0):
#       w = np.random.randn(D, C) # weights b/w I/O
#       params0 = [w]
#     elif (hidden_layer==1):
#       v = np.random.randn(D, self.M) # weights b/w I/first layer
#       w = np.random.randn(self.M, C) # weights b/w first/second layer
#       params0 = [v, w]
#     elif (hidden_layer==2):
#       v = np.random.randn(D, self.M) # weights b/w I/first layer
#       r = np.random.randn(self.M, self.M) # weights b/w first/second layer
#       w = np.random.randn(self.M, C) # weights b/w second/O layer
#       params = [v, r, w]
#     else: 
#       print("hidden layer should be <= 2")
#       return None
#     # END: Condition to have 0, 1, 2 layers
#     self.params = optimizer.run(gradient[hidden_layer], x, y, params0)
#     return self
    
#   def predict(self, x): # Tweak this method too
#     if(self.layers==0):
#       w = self.params
#       yh = softmax(np.dot(x, w))
#       return None
#     elif(self.layers==1):
#       v, w = self.params
#       z = logistic(np.dot(x, v)) # N x M
#       yh = logistic(np.dot(z, w)) # N x C
#     elif(self.layers==2):
#       v, r, w = self.params
#       h1 = logistic(np.dot(x, v))
#       h2 = logistic(np.dot(h1, r))
#       yh = softmax(np.dot(h2, w))
#     else:
#       print("Error")
#       return None
#     print(yh[:10])
#     y_pred = [np.argmax(yh[i, :]) for i in range(x.shape[0])]
#     return y_pred

In [11]:
class MLP:
  def __init__ (self, input_neurons, output_neurons, epochs=int(1e3), hidden_layer=1,
               hidden_neurons=128,learning_rate=1e-2, epsilon=1e-3, activation="sigmoid"):
    self.epochs = epochs
    self.hidden_layer = hidden_layer
    self.input_neurons = input_neurons
    self.hidden_neurons = hidden_neurons
    self.output_neurons = output_neurons
    self.learning_rate = learning_rate
    self.epsilon=epsilon
    self.activation=activation
    # no. of weights dependent on hidden_layers
    self.weights = self.init_weights(self.hidden_layer)

  def init_weights(self, hidden_layer):
    if(hidden_layer==0):
      w = np.random.randn(self.input_neurons, self.output_neurons) * np.sqrt(1./self.output_neurons)
      return np.asarray(w)
    elif (hidden_layer==1):
      v = np.random.randn(self.input_neurons, self.hidden_neurons) * np.sqrt(1./self.hidden_neurons)
      w = np.random.randn(self.hidden_neurons, self.output_neurons) * np.sqrt(1./self.output_neurons)
      return np.asarray([v,w])
    elif (hidden_layer==2):
      v = np.random.randn(self.input_neurons, self.hidden_neurons) * np.sqrt(1./self.hidden_neurons)
      r = np.random.randn(self.hidden_neurons, self.hidden_neurons) * np.sqrt(1./self.hidden_neurons)
      w = np.random.randn(self.hidden_neurons, self.output_neurons) * np.sqrt(1./self.output_neurons)
      return np.asarray([v,r,w])
    else:
      print("# of hidden layer should be <= 2")
      return None

  def fit(self, x_train, y_train):
    y_class = np.zeros((y_train.shape[0], np.max(y_train)+1))
    for i in range(y_train.shape[0]): y_class[i,y_train[i]] = 1
    
    norm = np.array([np.inf])
    t=0
    while(np.any(norm>self.epsilon) and t<self.epochs):
      # pdb.set_trace()
      yh = self.forward_pass(x_train)
      dweights = self.back_prop(yh, y_class)
      # print(dweights)
      self.update_weights(np.asarray(dweights))
      norm = np.array([np.linalg.norm(g) for g in dweights])
      print("Epoch: ", t, " with accuracy of: ", self.eval_acc(np.argmax(yh, axis=-1), y_train))
      print("Norm: ", norm)
      t+=1

  def activation_function(self, z, derivative=False):
    if(self.activation=="sigmoid"):
      if derivative: return sigmoid(z)*(1-sigmoid(z))
      else: return sigmoid(z)
    elif(self.activation=="tanh"):
      if derivative: return 1-np.square(tanh(z))
      else: return tanh(z)
    elif(self.activation=="relu"):
      if derivative: return 1 * (z>0)
      else: return relu(z)
    elif(self.activation=="leaky-relu"):
      if derivative:
        return 1 if (z>0) else 0.1
      else: return leaky_relu(z)
    else:
      print("invalid activation function")
      return None
  def forward_pass(self, x_train):
    f1 = x_train # N x D
    self.f_params = []
    if(self.hidden_layer==0):
      # input --> output
      f2 = np.dot(f1, self.weight[0]) # N x C
      f3 = softmax(f2) # N x C
      self.f_params = [f1, f2, f3]
      return f3

    elif (self.hidden_layer==1):
      # input --> first layer
      f2 = np.dot(f1, self.weights[0]) # N x M
      f3 = self.activation_function(f2) # N x M

      # first layer --> output
      f4 = np.dot(f3, self.weights[1]) # N x C
      f5 = softmax(f4) # N x C
      # print("f1", f1[:3])
      # print("f2", f2[:3])
      # print("f3", f3[:3])
      # print("f4", f4[:3])
      # print("f5", f5[:3])
      self.f_params = [f1, f2, f3, f4, f5]
      return f5

    elif (self.hidden_layer==2):
      # input --> first layer
      f2 = np.dot(f1, self.weight[0]) # N x M
      f3 = self.activation_function(f2) # N x M

      # first layer --> second layer
      f4 = np.dot(f3, self.weight[1]) # N x M
      f5 = self.activation_function(f4) # N x M

      # second layer --> output
      f6 = np.dot(f5, self.weight[2]) # N x C
      f7 = softmax(f6) # N x C
      self.f_params = [f1, f2, f3, f4, f5, f6, f7]
      return f7
    
    else:
      print("No forward pass")
      return None

  
  def back_prop(self, y_pred, y_train):
    N = y_pred.shape[0]
    b1 = y_pred-y_train # N x C
    # print('y_pred', y_pred[:3])
    # print('p_train', y_train[:3])
    if(self.hidden_layer==0):
      f1, f2, f3 = self.f_params

      e1 = b1 * softmax_diff(f2) # N x C
      b2 = np.dot(f1.T, e1)/N # D x C

      return b2

    elif (self.hidden_layer==1):
      f1, f2, f3, f4, f5 = self.f_params
      # print("b1", b1[:3])
      # print("f4", f4[:3])
      e1 = b1 * softmax_diff(f4) # N x C
      b2 = np.dot(f3.T, e1)/N # M x C
      # print("e1", e1[:3])
      # print("f3", f3[:3])
      # print("b2", b2[:3])
      b3 = np.dot(e1, self.weights[1].T) # N x M

      e2 = self.activation_function(f2, derivative=True)
      b4 = np.dot(f1.T, b3*e2)/N # D x M
      # print("b4", b4[:3])
      # print("b2", b2[:3])
      return [b4, b2]

    elif (self.hidden_layer==2):
      f1, f2, f3, f4, f5, f6, f7 = self.f_params

      e1 = b1 * softmax_diff(f6) # N x C
      b2 = np.dot(f5.T, e1)/N # M x C

      b3 = np.dot(e1, self.weights[2].T) # N x M

      e2 = self.activation_function(f4, derivative=True) # N x M
      b4 = np.dot((b3*e2).T, f3)/N # M x M

      b5 = np.dot(b3*e2, self.weights[1].T) # N x M

      e3 = self.activation_function(f2, derivative=True) # N x M
      b6 = np.dot(f1.T, (b5*e3))/N # D x M

      return [b6, b4, b2]
    
    else:
      print("No backpropagation")
      return None
  
  def update_weights(self, dweights):
    self.weights -= self.learning_rate * dweights

  def predict(self, x_test):
    yh = self.forward_pass(x_test)
    return np.argmax(yh, axis=-1)

  def eval_acc(self, y_pred, y_test):
    acc = y_pred==y_test
    # print("Prediction: 0 --> %d ; 1 --> %d ; 2 --> %d ; 3 --> %d ; 4 --> %d ; 5 --> %d ; 6 --> %d ; 7 --> %d ; 8 --> %d ; 9 --> %d"  % (np.sum(y_pred == 0), np.sum(y_pred==1), np.sum(y_pred==2), np.sum(y_pred==3), np.sum(y_pred==4), np.sum(y_pred==5), np.sum(y_pred==6), np.sum(y_pred==7), np.sum(y_pred==8), np.sum(y_pred==9)))
    return np.sum(acc)/len(y_test)
    # return(np.mean((y_pred==y_test)))



## Activation Function
Sigmoid, ReLu, Leaky ReLu, tanh for intermediate layer 

Softmax and derivative for classification and gradient update

In [12]:
sigmoid = lambda z: 1./ (1 + np.exp(-z))
# sigmoid_diff = lambda z: sigmoid(z)*(1-sigmoid(z))

relu = lambda z: np.maximum(0, z)

tanh = lambda z: np.tanh(z)

leaky_relu = lambda z: np.maximum(0.1*z, z)

softmax = lambda z: np.exp(z-z.max()) / np.sum(np.exp(z-z.max()), axis = -1)[:, None]
softmax_diff = lambda z: softmax(z)*(1-softmax(z))


In [13]:
# def eval_acc(y_pred, y_real):
#   if(y_pred.ndim==1 and y_real.ndim==1): 
#     # print(y_pred[:5])
#     # print(y_real[:5])
#     print(np.max(y_pred))
#     print(np.sum(y_pred==y_real))
#     print(len(y_pred))
#     return np.sum(y_pred==y_real)/len(y_pred)
#   else: 
#     print("dimension should be 1")
#     return None

## Gradient Calculation

In [14]:
# class GradientDescent:
    
#   def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8):
#     self.learning_rate = learning_rate
#     self.max_iters = max_iters
#     self.epsilon = epsilon
        
#   def run(self, gradient_fn, x, y, params):
#     norms = np.array([np.inf])
#     t = 1
#     while np.any(norms > self.epsilon) and t < self.max_iters:
#       # add parameter of number of layers
#         grad = gradient_fn(x, y, params)
#         for p in range(len(params)):
#             params[p] -= self.learning_rate * grad[p]
#         t += 1
#         norms = np.array([np.linalg.norm(g) for g in grad])
#         print("%d epoch completed with norm of:" %(t))
#         print(norms)
#     return params

# Main Body of Code
## Importing Data and normalization

In [15]:
mnist_dataset = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

print("Train image shape: ", train_images.shape)
print("Train label shape: ", train_labels.shape)
print("\nTest image shape: ", test_images.shape)
print("Test label shape: ", test_labels.shape)

train_images = train_images/255
test_images = test_images/255

fraction = 0.10 # 10% validation and 90% training set
n_train = train_images.shape[0]
# w, h = train_images[0].shape
train_images = train_images[:int(fraction*n_train)]
train_labels = train_labels[:int(fraction*n_train)]
train_images = train_images.reshape(train_images.shape[0], -1)
print(train_images.shape)

n_train = train_images.shape[0]
x_test = train_images[:int(fraction*n_train)]
y_test = train_labels[:int(fraction*n_train)]
print(x_test.shape)

x_train = train_images[int(fraction*n_train):]
y_train = train_labels[int(fraction*n_train):]
print(x_train.shape)
# plt.imshow(train_images[10], cmap='gray')
# plt.colorbar()
# plt.grid(False)
# plt.show()

Train image shape:  (60000, 28, 28)
Train label shape:  (60000,)

Test image shape:  (10000, 28, 28)
Test label shape:  (10000,)
(6000, 784)
(600, 784)
(5400, 784)


In [16]:
mlp = MLP(x_test.shape[1], np.max(y_test)+1, epsilon=1e-3, activation="relu")
# pdb.set_trace()
mlp.fit(x_train, y_train)
yh = mlp.predict(x_test)

# optimizer = GradientDescent(learning_rate=1e-2, max_iters=1e3)

# yh = mlp.fit(x_train, y_train, optimizer, hidden_layer=1).predict(x_test)
# print("\n\nAccuracy of validation set is: %.3f\n\n" % eval_acc(np.array(yh), np.array(y_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Epoch:  0  with accuracy of:  0.029444444444444443
Norm:  [2.01736851 0.63059389]
Epoch:  1  with accuracy of:  0.04111111111111111
Norm:  [1.94507432 0.61490543]
Epoch:  2  with accuracy of:  0.0512962962962963
Norm:  [1.84178305 0.59307266]
Epoch:  3  with accuracy of:  0.05925925925925926
Norm:  [1.73541934 0.57111063]
Epoch:  4  with accuracy of:  0.06814814814814815
Norm:  [1.63846018 0.55104846]
Epoch:  5  with accuracy of:  0.07388888888888889
Norm:  [1.55278192 0.53243958]
Epoch:  6  with accuracy of:  0.07759259259259259
Norm:  [1.47516827 0.51437106]
Epoch:  7  with accuracy of:  0.08203703703703703
Norm:  [1.40404983 0.49622572]
Epoch:  8  with accuracy of:  0.08481481481481482
Norm:  [1.33760809 0.47774457]
Epoch:  9  with accuracy of:  0.08981481481481482
Norm:  [1.27358382 0.45900095]
Epoch:  10  with accuracy of:  0.09314814814814815
Norm:  [1.21344565 0.44027252]
Epoch:  11  with accuracy of:  0.09574074074074074
Norm:  [1.15581457 0.42187796]
Epoch:  12  with accuracy 